In [1]:
# ============================================================================
# SECTION 1: LANGFUSE CONFIGURATION
# ============================================================================

import os

# Set Langfuse credentials directly
os.environ["LANGFUSE_SECRET_KEY"] = "****"
os.environ["LANGFUSE_PUBLIC_KEY"] = "****"
os.environ["LANGFUSE_BASE_URL"] = "****"

print("✅ Langfuse credentials configured!")

✅ Langfuse credentials configured!


In [3]:
# ============================================================================
# SECTION 2: IMPORTS AND CONFIGURATION
# ============================================================================

import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
import pandas as pd
import pymysql
from getpass import getpass
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import Langfuse
from langfuse.callback import CallbackHandler

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [4]:
from langchain_community.utilities import SQLDatabase
import urllib.parse

db_user = "****"
db_password = urllib.parse.quote_plus("****")  # URL-encode password
db_host = "****"
db_name = "****"

try:
    db = SQLDatabase.from_uri(
        f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
        sample_rows_in_table_info=3
    )
    print("✅ Successfully connected to the database!")

    # Display table names
    tables = db.get_usable_table_names()
    print("📋 Tables in the database:")
    for t in tables:
        print("-", t)

except Exception as e:
    print("❌ Failed to connect to the database.")
    print("Error:", e)

✅ Successfully connected to the database!
📋 Tables in the database:
- discounts
- t_shirts


In [5]:
print(db.get_table_info(["t_shirts"]))


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	XS	25	97
2	Van Huesen	Red	S	39	72
3	Van Huesen	Red	M	21	43
*/


In [6]:
print(db.get_table_info(["discounts"]))


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langfuse.callback import CallbackHandler

# Initialize Langfuse callback handler
langfuse_handler = CallbackHandler()

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4, api_key="****")

# Toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Agent with Langfuse callback
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools"   # required for GPT-4o
)

# Ask a question with Langfuse tracking
response = agent.invoke(
    {"input": "How many t-shirts are in stock?"},
    config={"callbacks": [langfuse_handler]}
)
print(response["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mdiscounts, t_shirts
Invoking: `sql_db_schema` with `{'table_names': 't_shirts'}`



CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	XS	25	97
2	Van Huesen	Red	S	39	72
3	Van Huesen	Red	M	21	43

Invoking: `sql_db_query_checker` with `{'query': 'SELECT SUM(stock_quantity) AS total_stock FROM t_shirts;'}`


```sql
SELECT SUM(stock_quantity) AS total_stock FROM t_shirts;

Invoking: `sql_db_query` with `{'query': 'SELECT SUM(

In [8]:
response = agent.invoke(
    {"input": "How much is the price of the inventory for all small size tshirts"},
    config={"callbacks": [langfuse_handler]}
)
print(response["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mdiscounts, t_shirts
Invoking: `sql_db_schema` with `{'table_names': 't_shirts'}`



CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	XS	25	97
2	Van Huesen	Red	S	39	72
3	Van Huesen	Red	M	21	43
*/
Invoking: `sql_db_schema` with `{'table_names': 'discounts'}`



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	

In [9]:
response = agent.invoke(
    {"input": "If i Sell all my Levi tshirt today by applying discounts how much revenue will be generated?"},
    config={"callbacks": [langfuse_handler]}
)
print(response["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mdiscounts, t_shirts
Invoking: `sql_db_schema` with `{'table_names': 'discounts'}`



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/
Invoking: `sql_db_schema` with `{'table_names': 't_shirts'}`



CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NO

In [10]:
response = agent.invoke(
    {"input": "How many white color levi tshirts we have"},
    config={"callbacks": [langfuse_handler]}
)
print(response["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mdiscounts, t_shirts
Invoking: `sql_db_schema` with `{'table_names': 't_shirts'}`



CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	XS	25	97
2	Van Huesen	Red	S	39	72
3	Van Huesen	Red	M	21	43

Invoking: `sql_db_query_checker` with `{'query': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"}`


```sql
SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'

Invoking: `sq

In [11]:
import pandas as pd
import datetime
from IPython.display import display
from openpyxl import load_workbook  # Required for appending Excel files

# Initialize Langfuse callback handler for this cell
langfuse_handler = CallbackHandler()

# -----------------------------
# Step 1: User input
# -----------------------------
user_prompt = input("Enter your query (e.g., 'How many red Levi t-shirts?'): ")

# -----------------------------
# Step 2: Run through SQL Agent with Langfuse tracking
# -----------------------------
response_data = agent.invoke(
    {"input": user_prompt},
    config={"callbacks": [langfuse_handler]}
)
response = response_data["output"]

# Extract raw SQL result
try:
    if isinstance(response, list):
        df = pd.DataFrame(response)
    elif isinstance(response, pd.DataFrame):
        df = response.copy()
    else:
        df = pd.DataFrame({"Result": [response]})
except Exception as e:
    print("❌ Error converting result to DataFrame:", e)
    df = pd.DataFrame({"Result": [response]})

# -----------------------------
# Step 3: Add metadata
# -----------------------------
df["User Prompt"] = user_prompt
df["Timestamp"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# -----------------------------
# Step 4: Append to Excel
# -----------------------------
excel_file = "query_result.xlsx"

try:
    # Try to open existing file
    book = load_workbook(excel_file)
    with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        df.to_excel(writer, sheet_name="Results", index=False, header=False, startrow=writer.sheets["Results"].max_row)
    print(f"✅ Result appended to {excel_file}")
except FileNotFoundError:
    # If file does not exist, create it
    with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="Results", index=False)
    print(f"✅ Excel file created: {excel_file}")

# -----------------------------
# Step 5: Display the DataFrame
# -----------------------------
display(df)

Enter your query (e.g., 'How many red Levi t-shirts?'):  what do we mainly sell




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mdiscounts, t_shirts
Invoking: `sql_db_schema` with `{'table_names': 't_shirts'}`



CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	XS	25	97
2	Van Huesen	Red	S	39	72
3	Van Huesen	Red	M	21	43
*/
Invoking: `sql_db_schema` with `{'table_names': 'discounts'}`



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	

,Result,User Prompt,Timestamp
0,The database contains two main tables: `t_shir...,what do we mainly sell,2025-11-21 09:13:33
